<a href="https://colab.research.google.com/github/VinaySingh561/LARGE-LANGUAGE-MODELS/blob/main/LLM_tokenizer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step:1 Creating token from words

Refer to the following link for downloaing 'verdict' file [Verdict](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/the-verdict.txt)

In [1]:
with open("the-verdict.txt","r",encoding='utf-8') as f:
  raw_text = f.read()

print("total number of characters", len(raw_text))
print(raw_text[:99])

total number of characters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re
text = "Hello, my name is vinay singh. What is your name?"
result = re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'my', ' ', 'name', ' ', 'is', ' ', 'vinay', ' ', 'singh.', ' ', 'What', ' ', 'is', ' ', 'your', ' ', 'name?']


In [3]:
# but in the above line comma and ? etc. are not seperated
result = re.split(r'([.,]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'my', ' ', 'name', ' ', 'is', ' ', 'vinay', ' ', 'singh', '.', '', ' ', 'What', ' ', 'is', ' ', 'your', ' ', 'name?']


In [4]:
# now we have to get rid of the white spaces
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'my', 'name', 'is', 'vinay', 'singh', '.', 'What', 'is', 'your', 'name?']


In [5]:
## we need to remove some other special symbols
text = "Hello, what do you mean!, I don't get it. Is this-- a test?"
result = re.split(r'([,:;?!@#$%^&*()]|-|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)


['Hello', ',', 'what', 'do', 'you', 'mean', '!', ',', 'I', "don't", 'get', 'it.', 'Is', 'this', '-', '-', 'a', 'test', '?']


In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
print(len(preprocessed))

4690


### Step2 : Creating Token Ids

In [8]:
all_words = sorted(set(preprocessed))
print(len(all_words))

1130


In [9]:
vocab = {token:integer for integer, token in enumerate(all_words)}

In [10]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [11]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}


  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed = [
        item.strip() for item in preprocessed if item.strip()
    ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[id] for id in ids])
    ## replace spaces before the specified punctuations
    text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
            Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)



[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [14]:
## if some word is not present in the vacab
text = "Hello"
ids = tokenizer.encode(text)
print(ids)

KeyError: 'Hello'

### Adding special context tokens

In [51]:
# for unknown words present in the text
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<end>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [52]:
list(vocab.items())[-5:]

[('younger', 1127),
 ('your', 1128),
 ('yourself', 1129),
 ('<end>', 1130),
 ('<|unk|>', 1131)]

In [53]:
### Simpletokenizer will also change accordingly
class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {integer:text for text,integer in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    # now removing the whitespace
    preprocessed = [
        item.strip() for item in preprocessed if item.strip()
    ]
    # adding unknown and end of text tokens
    preprocessed = [
        item if item in self.str_to_int
        else "<|unk|>" for item in preprocessed
    ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[id] for id in ids])
    ## replace spaces before the specified punctuations
    text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text



In [54]:
text1 = "Hello, do you know me?"
text2 = "In the sunlit terraces of the palace."

text = " <end> ".join((text1,text2))
print(text)

Hello, do you know me? <end> In the sunlit terraces of the palace.


In [55]:
tokenizer = SimpleTokenizerV2(vocab)
ids = tokenizer.encode(text)
print(ids)

[1131, 5, 355, 1126, 596, 663, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [56]:
tokenizer.decode(ids)

'<|unk|>, do you know me? <end> In the sunlit terraces of the <|unk|>.'